# 1. QCFractal collection creation

The Binding Energy Evaluation Platform (BEEP) is powered by QCFractal meaning that all the data is saved 
in easy-to-query data collections. You can check out the QCFractal documentation on how to install
and initialize a  [QCFractal server](http://docs.qcarchive.molssi.org/projects/QCFractal/en/stable/quickstart.html). For this demonstration 
we will  use a temporary "Snowflake" sever. So as a first step we import the QCfractal module and  start our toy-server.

In [ ]:
from qcfractal import FractalSnowflakeHandler
import qcfractal.interface as ptl

The second library is an interface with the QCPortal libraries that is the front end to the server data. Once the libraries are imported we can initialize a server client. 

In [ ]:
client = FractalSnowflakeHandler().client()
client

Alternatively you can spin up your own permanent server using the `qcfractal-server` command line tool. Instructions on how to initialize a server can be found in the [QCFractal server documentatnion](http://docs.qcarchive.molssi.org/projects/QCFractal/en/stable/setup_server.html)

Now that our client is loaded we will create two molecule objects that we will save in data collections called OptimizationDatasets. We will create the molecules using a string of xyz coordinates. The molecules we will use for this example are a water pentamer and the methanol molecule. 

In [ ]:
w5_xyz = '''H                 0.172970266612     1.380092911130     0.457386526990
    H                -1.911743135806     0.591094283257    -0.435363356758
    O                -0.689370829428     1.203413826146     0.902232866763
    O                 1.683383001712     1.425574187566    -0.342538154484
    H                 1.982201397053     0.492649561018    -0.410548565072
    H                 1.705903260790     1.780383434343    -1.232746014340
    O                 2.115028368559    -1.237273923747    -0.530994509997
    H                 2.756214454001    -1.798495919214    -0.094310909878
    H                 1.223063867482    -1.530410584825    -0.243491125950
    O                -0.414534335596    -1.638505160179     0.377481322124
    H                -1.086606563218    -1.481629097791    -0.302673057926
    H                -0.496166832469    -0.853568312395     0.936120264347
    O                -2.243895804335    -0.047924230121    -1.085237431327
    H                -3.197163055178     0.039720230017    -1.107188531747
    H                -0.777303421514     1.838006282802     1.614357402995
'''

In [ ]:
ch3oh_xyz ='''C                 0.741964933916     0.010920665036     0.000024437675
    O                -0.674528191722    -0.047141542148    -0.000053676759
    H                -1.021577773322     0.845691219101     0.000133758184
    H                 1.125169842343     0.512990825971    -0.888963972862
    H                 1.097404033083    -1.014140879059    -0.000355642164
    H                 1.125094706522     0.512309796358     0.889429438634
'''

With these xyz we can now create a molecule object: 

In [ ]:
mol_w5 = ptl.Molecule.from_data(w5_xyz)
mol_ch3oh = ptl.Molecule.from_data(ch3oh_xyz)

Now we can create an OptimizationDataset and add those molecules as initial molecules. First we create two collection one for adding small water clusters, and a second one

In [ ]:
ds_w5 = ptl.collections.OptimizationDataset("pentamer_collection", client=client)
ds_w5.save()
ds_smol = ptl.collections.OptimizationDataset("small_molecules", client=client)
ds_smol.save()

Here, we named the collection that will store the water pentamers "pentamer collection" and the collection with with the small molecules is called "small_molecules". It is important to save after you created the collection. You can check if the OptimiazationDataset was created correctly using the `list_collection` method:

In [ ]:
client.list_collections()

Finally we add the molecule to the OptimizationDataSet called `pentamer_collection`

In [ ]:
ds_w5.add_entry("w5_1", mol_w5)

The same can be done for the ch3oh molecule:

In [ ]:
ds_smol.add_entry("ch3oh", mol_ch3oh)

Now you cloud add more isomers of the water pentamer or other small molecules to that same collection. These collections are the starting point for the sampling procedure since the `launch_sampling` commandline tool
draws  molecules from these collections and samples all the molecules that are present in the collection with the water clusters. Their name must be specified as one of the options of the program. 
In order to optimzize the collection you will need to start a `qcfractal-manager` that handles the computations
submited to the queue. For more details about how to start a manager check out the [QCFractal manager documentation](http://docs.qcarchive.molssi.org/projects/QCFractal/en/stable/managers.html)

##  Ice-mantle surface of a set of W_12

As a final exercise we will build a collection with water clusters containing 12 molecules that have 
been amorphized using AIMD. We will usa a span of 5 water clusters each containing 12 molecules that will represent different segments of a ice-mantle surface of an interstellar grain. The xyz structures are contained in a file within this folder called `w12.dat`. We will read in the 5 structures and save them in a dataset. 

In [5]:
w12 = open("w12.dat", 'r').read()
w12_mols = w12.split('\n\n')

Now we create a new collections where the set of water clusters will be saved and add each of the 5 structures
contained in the file and save it. 

In [8]:
ds_w12 = ptl.collections.OptimizationDataset("water_12", client=client)

for i in range(len(w12_mols)):
    ds_w12.add_entry("w12_"+str(i+1), w12_mols[i])
    ds_w12.save()

5


We can use this small model of an amorphous solid water surface to compute a binding energy of 
a small astrophysically relevant molecule. We will do this in the second tutorial! 